In [41]:
pip install pandas numpy nltk scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [42]:
import string
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [43]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/acrymonic/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [44]:
df = pd.read_csv('spam_ham_dataset.csv')

In [45]:
df['text'] = df['text'].apply(lambda x: x.replace('\r\n', ' '))

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5171 non-null   int64 
 1   label       5171 non-null   object
 2   text        5171 non-null   object
 3   label_num   5171 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 161.7+ KB


In [47]:
#remove punctuation, stemming process
stemmer = PorterStemmer()
corpus = [] #transformed email

stopwords_set = set(stopwords.words('english'))

for i in range(len(df)):
    text = df['text'].iloc[i].lower()
    text = text.translate(str.maketrans('','',string.punctuation)).split()
    #remove punctuation (first two params are for translating abc, xyz a mapped to x, b to y and so on)
    text = [stemmer.stem(word) for word in text if word not in stopwords_set]
    text = ' '.join(text)
    corpus.append(text)

In [48]:
##print(corpus[0])
##print("==============================")
##print(df.text.iloc[0])

In [49]:
#vectorize 
vectorizer = CountVectorizer()

x = vectorizer.fit_transform(corpus).toarray()
y = df.label_num

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [50]:
clf = RandomForestClassifier(n_jobs=-1)
clf.fit(x_train, y_train)

RandomForestClassifier(n_jobs=-1)

In [51]:
clf.score(x_test, y_test)

0.9748792270531401

In [82]:
#email_to_classify = df.text.values[10]
def spam_emaildetector():
    inputint = int(input("Enter an index from the database (0-5170):"))
    if (inputint<0 or inputint>5170):
        print("INVALID")
        return
    else:
        email_to_classify = df.text.values[inputint]
        email_text = email_to_classify.lower().translate(str.maketrans('', '', string.punctuation)).split()
        email_text = [stemmer.stem(word) for word in email_text if word not in stopwords_set]
        email_text = ' '.join(email_text)

        email_corpus = [email_text]

        x_email = vectorizer.transform(email_corpus)

        finval = clf.predict(x_email)
        if finval[0] == 0:
            print("Your email is not spam")
        if finval[0] == 1:
            print("Your email is spam")

In [83]:
spam_emaildetector()

Enter an index from the database (0-5170): 3


Your email is spam


In [70]:
df #database used from training.

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291 thi...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001 ( see at...",0
2,3624,ham,"Subject: neon retreat ho ho ho , we ' re aroun...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs this deal is to b...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft the transport vo...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms hpl c...,0
5168,2933,ham,Subject: calpine daily gas nomination > > juli...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0
